<a href="https://colab.research.google.com/github/NQT2906/CS114.K21/blob/Case_Study/Sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clone data from git**

In [1]:
!git clone -b Case_Study 'https://github.com/NQT2906/CS114.K21.git' Dataset

fatal: destination path 'Dataset' already exists and is not an empty directory.


# Mô tả dữ liệu
- Bộ dữ liệu được thu thập từ 2 website: TheOnion (trang này bao gồm các tin tức châm biếm về các sự kiện nổi bật đang diễn ra gần đây, tác giả đã thu thập các tiêu đề từ các tin tức và gán nhãn cho nó là tin sarcsm) và HuffPost là bao gồm các tin không châm biến.
- Ưu diểm của bộ dữ liệu này sao với các bộ dữ liệu Twitter: mục đích duy nhât của TheOnion là xuất bản các tin tức châm biếm cho nên sẽ có được các nhãn chất lượng hơn và ít độ nhiễu hơn.
- Mỗi dòng dữ liệu gồm có 3 thuộc tính: is_sarcsm (1 nếu tin tức đó là châm biếm và ngược lại là 0)
- Nhiệm vụ: Xác định các câu châm biếm, phát hiện giữa các tin giả và tin hợp pháp

# **Import library**

In [2]:
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Read and combine data**

- Dữ liệu ban đầu gồm 2 file .json
- Tiến hành đọc từng file và lưu dưới dạng dataframe với thư viện pandas

In [3]:
dataset_1 = pd.read_json('https://raw.githubusercontent.com/NQT2906/CS114.K21/Sarcasm/Sarcasm/Sarcasm_Headlines_Dataset.json', lines = True)
dataset_2 = pd.read_json('https://raw.githubusercontent.com/NQT2906/CS114.K21/Sarcasm/Sarcasm/Sarcasm_Headlines_Dataset_v2.json', lines = True)

In [4]:
dataset_1

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
26704,https://www.huffingtonpost.com/entry/american-...,american politics in moral free-fall,0
26705,https://www.huffingtonpost.com/entry/americas-...,america's best 20 hikes,0
26706,https://www.huffingtonpost.com/entry/reparatio...,reparations and obama,0
26707,https://www.huffingtonpost.com/entry/israeli-b...,israeli ban targeting boycott supporters raise...,0


In [5]:
is_sar = dataset_2.iloc[:, 0]
art = dataset_2.iloc[:, 2]

dataset_2.iloc[:, 0] = art
dataset_2.iloc[:, 2] = is_sar

dataset_2.columns = ['article_link', 'headline', 'is_sarcastic']

In [6]:
dataset_2

,article_link,headline,is_sarcastic
0,https://www.theonion.com/thirtysomething-scien...,thirtysomething scientists unveil doomsday clo...,1
1,https://www.huffingtonpost.com/entry/donna-edw...,dem rep. totally nails why congress is falling...,0
2,https://www.huffingtonpost.com/entry/eat-your-...,eat your veggies: 9 deliciously different recipes,0
3,https://local.theonion.com/inclement-weather-p...,inclement weather prevents liar from getting t...,1
4,https://www.theonion.com/mother-comes-pretty-c...,mother comes pretty close to using word 'strea...,1
...,...,...,...
28614,https://www.theonion.com/jews-to-celebrate-ros...,jews to celebrate rosh hashasha or something,1
28615,https://local.theonion.com/internal-affairs-in...,internal affairs investigator disappointed con...,1
28616,https://www.huffingtonpost.com/entry/andrew-ah...,the most beautiful acceptance speech this week...,0
28617,https://www.theonion.com/mars-probe-destroyed-...,mars probe destroyed by orbiting spielberg-gat...,1


In [7]:
dataset = pd.concat([dataset_1, dataset_2])

In [8]:
dataset

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
28614,https://www.theonion.com/jews-to-celebrate-ros...,jews to celebrate rosh hashasha or something,1
28615,https://local.theonion.com/internal-affairs-in...,internal affairs investigator disappointed con...,1
28616,https://www.huffingtonpost.com/entry/andrew-ah...,the most beautiful acceptance speech this week...,0
28617,https://www.theonion.com/mars-probe-destroyed-...,mars probe destroyed by orbiting spielberg-gat...,1


In [9]:
dataset = dataset.drop_duplicates()

In [10]:
dataset

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
28497,https://www.theonion.com/tyson-holds-contest-t...,tyson holds contest to let fans submit new ide...,1
28509,https://politics.theonion.com/increasingly-coc...,increasingly cocky bernie sanders announces he...,1
28520,https://www.theonion.com/cash-strapped-zuckerb...,cash-strapped zuckerberg forced to sell 11 mil...,1
28544,https://local.theonion.com/grocery-store-bar-a...,grocery store bar actually has great little ha...,1


- Input: là các headline tương ứng với cột ở vị trí 1 trong dataset.
- Ouput: là giá trị xác định có phải là tin mỉa mai hay không

In [11]:
X = dataset.iloc[:, 1].values
Y = dataset.iloc[:, 2].values

# **Preprocessing**

- Một số thư viện làm sạch dữ liệu:
1. Thư viện re cho loại bỏ ký tự đặc biệt.
2. Thư viện nltk.corpus.stopwords cho loại bỏ các từ không có nhiều ý nghĩa.
3. Thư viện nltk.stem.porter.PorterStemmer cho việc chuyển các từ về dạng nguyên mẫu.


In [12]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
 def create_corpus(X):    
    corpus = []
    for review in X: # Duyệt qua từng phần tử trong X_train.values
        review = re.sub('[^a-zA-Z]', ' ', review) #loại bỏ tất cả bình luận không phải chữ cái
        review = review.split() #chuyển chuỗi thành các tin tức riêng biệt
        ps = PorterStemmer()
        #loại bỏ các từ không có ý nghĩa trong câu
        #chuyển về dạng nguyên mẫu
        review = [ps.stem(word) for word in review if word not in stopwords.words('english')] 
        review = ' '.join(review) # Ghép các từ sau khi xử lý lại thành 1 câu
        #thêm bình luận đã qua xử lý vào corpus
        corpus.append(review)
    return corpus

In [14]:
 corpus = create_corpus(X)

# **Create dictionary for word in train and test**

- Các thuật toán Machine Learning chỉ xử lý được dữ liệu ở dạng số.
- Bag of worlds là một mô hình biễu diễn từ và câu trong NLP.
Mỗi từ trong mô hình bag of words được biểu diễn bằng một vector:
- Có kích thước (số lượng phần tử) bằng với số lượng từ trong từ điển
- Có duy nhất một phần tử là 1, các phần tử còn lại là 0.
- Phần tử 1 nằm ở vị trí bằng với vị trí của từ đó trong từ điển.
- Việc chuyển các câu trong tập văn bản thành vector dựa trên mô hình Bag of words được thực hiện bởi lớp CountVectorizer trong module sklearn.feature_extraction.text.

In [15]:
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()
trainVocab = cv.vocabulary_ #export tập từ vựng

In [16]:
np.save('dictionary.npy', trainVocab)

# **Split data to train and test (0.8 train and 0.2 test)**

- Chia dữ liệu theo tỷ lệ 8:2 (80% dữ liệu huấn luyện) và 20% dữ liệu kiểm tra
- Sử dụng 2 thuộc tính headline và is_sarcasm để huấn luyện mô hình
- Biến độc lập là headline và biến phụ thuộc là is_sarcasm

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, random_state = 0)

In [ ]:
cv = CountVectorizer(max_features = 10000)
X_train = cv.fit_transform(corpus_train).toarray()
trainVocab = cv.vocabulary_ #export tập từ vựng

In [ ]:
cv = CountVectorizer(vocabulary=trainVocab)
X_test = cv.fit_transform(corpus_test).toarray()

In [ ]:
print(X_train.shape)
print(Y_train.shape)

In [ ]:
print(X_test.shape)
print(Y_test.shape)

# **Choose model**

- Sử dụng mô hình Naïve Bayes

- Bài toán đặt ra ở đây là xác định xác suất để để một headline thuộc về class 1 (tin châm biếm) hay là 0. 
- Mục tiêu ở đây, ví dụ với một string là "Em ăn cơm chưa". Chúng ta sẽ tách string này thành từng từ riêng biệt: "Em", "ăn", "cơm", chưa". 
- Làm trương tự vậy với tất cả string. Và ta sẽ tính xác xuất trên tất cả các string với mỗi từ. Một cách đơn giản, là từ nào xuất hiện càng nhiều trong các tin nhắn được gán nhãn là 1 thì khi một tin nhắn chưa được gán nhãn chứa từ đấy. Xác suất nó là tin châm biếm càng cao.

In [19]:
model = MultinomialNB()
model.fit(X_train, Y_train)

Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

In [20]:
print("Score of model (MultinomialNB) in train set:", model.score(X_train, Y_train))
print("Score of model (MultinomialNB) in test set:", model.score(X_test, Y_test))

Score of model (MultinomialNB) in train set: 0.8781286856244267
Score of model (MultinomialNB) in test set: 0.7821453529000699


In [21]:
from sklearn.linear_model import LogisticRegression
model_log = LogisticRegression(random_state = 0)
model_log.fit(X_train, Y_train)

Y_train_pred = model_log.predict(X_train)
Y_test_pred = model_log.predict(X_test)

In [22]:
print("Score of model (LogisticRegression) in train set:", model_log.score(X_train, Y_train))
print("Score of model (LogisticRegression) in test set:", model_log.score(X_test, Y_test))

Score of model (LogisticRegression) in train set: 0.9166120648233085
Score of model (LogisticRegression) in test set: 0.7917540181691125


# **Fine Tuning**

- Tiến hành tuning model bằng GridSearchCV, chúng ta sẽ tiến hành thay đổi than số làm mịn Laplace
- Theo như mặc định của mô hình tham số làm mịn là 0 (tương ứng với không làm mịn)
- Kiếm tra kết quả bằng phương pháp đánh giá chéo K-Fold (k = 5)

In [23]:
from sklearn.model_selection import GridSearchCV

In [24]:
params = {'alpha': [0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4]}
clf = MultinomialNB()
clf = GridSearchCV(clf, params, cv = 5)
clf.fit(X_train, Y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                     fit_prior=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3,
                                   1.4]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [25]:
print(clf.best_params_)

{'alpha': 1.3}


In [26]:
best_clf = clf.best_estimator_
Y_train_pred = best_clf.predict(X_train)
print('Training size = %d, accuracy = %.2f%%' % \
      (X_train.shape[0],accuracy_score(Y_train, Y_train_pred)*100))

Training size = 22893, accuracy = 87.38%


In [27]:
best_clf = clf.best_estimator_
Y_test_pred = best_clf.predict(X_test)
print('Test size = %d, accuracy = %.2f%%' % \
      (X_test.shape[0],accuracy_score(Y_test, Y_test_pred)*100))

Test size = 5724, accuracy = 78.42%


# **Save model**

In [28]:
import pickle

In [29]:
filename = 'MultinomialNB_Sarcasm.sav'
pickle.dump(model, open(filename, 'wb'))

In [30]:
filename = 'Logistic_Regression_Sarcasm.sav'
pickle.dump(model_log, open(filename, 'wb'))

In [31]:
filename = 'Multi_Tuning_Sarcasm.sav'
pickle.dump(best_clf, open(filename, 'wb'))

# **Get new data**

- Dữ liệu được thu thập từ trang web TheOnion bằng cách sử dụng thư viện BeatifulSoup
- Dữ liệu gồm có 3991 dòng và 2 thuộc tính Headline và Sarcasm 
- Tất cả các dàng headline đều là tin chầm biếm cho nên sẽ được gán nhãn là 1

In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [ ]:
res1 = []

num = 1593005340413

In [ ]:
for i in range(300):
  #time = random.randint(10000000, 20000000)
  num = int(num) - 427919860
  url = 'https://www.theonion.com/latest' + '?startTime=' + str(num)
  r = requests.get(url)
  resp = r.text
  soup = BeautifulSoup(resp, features = 'html.parser')
  for headline in soup.find_all('h2', attrs = {'class': 'sc-759qgu-0 cYlVdn cw4lnv-6 eXwNRE'}):
    res1.append(headline.get_text())

In [ ]:
np.unique(res)

In [ ]:
data = pd.DataFrame(res, columns=['Headline'])

In [ ]:
data.to_csv('Headline_Case_study.csv')

# **Load model and test new data**

**Load model**

In [32]:
import pickle

model_log = pickle.load(open('/content/Logistic_Regression_Sarcasm.sav', 'rb'))
model_NB = pickle.load(open('/content/MultinomialNB_Sarcasm.sav', 'rb'))
model_nb_tunning = pickle.load(open('/content/Multi_Tuning_Sarcasm.sav', 'rb'))

**Load data in crawl file**

In [ ]:
datareal = pd.read_csv('/content/Dataset/Headline_Case_study.csv')

In [33]:
datareal = pd.read_csv('/content/Dataset/Headline_Case_study.csv')

X_real = datareal.iloc[:, 1].values
Y_real = datareal.iloc[:, 2].values

In [34]:
X_real.shape

(3935,)

In [ ]:
X_real = np.unique(X_real)

In [ ]:
X_real.shape

**Load dictionary in training set and convert it to dict**

In [35]:
dictionary_load = np.load('/content/dictionary.npy',allow_pickle='TRUE').item()

**Use CountVectorizer to fit_transform X_real**

In [36]:
corpus_real = create_corpus(X_real)

In [37]:
cv = CountVectorizer(vocabulary=dictionary_load)
X_real = cv.fit_transform(corpus_real).toarray()

In [38]:
print(X_real.shape)
print(Y_real.shape)


(3935, 17495)
(3935,)


**Predict and Evaluate**

In [ ]:
Y_real_pred_NB = model_NB.predict(X_real)

In [42]:
print("Score of model (MultinomialNB tuning) in real set:", model_nb_tunning.score(X_real, Y_real))

Score of model (MultinomialNB tuning) in real set: 0.5839898348157561


In [41]:
print("Score of model (MultinomialNB) in real set:", model_NB.score(X_real, Y_real))

Score of model (MultinomialNB) in real set: 0.5893265565438374


In [ ]:
Y_real_pred_Log = model_log.predict(X_real)

In [40]:
print("Score of model (Logistic) in real set:", model_log.score(X_real, Y_real))

Score of model (Logistic) in real set: 0.5514612452350699


# **Application**

In [ ]:
headline = input()
corpus_input = create_corpus(headline)
headline_input = cv.fit_transform(corpus_input).toarray()
res = best_clf.predict(headline_input)
if res.any() == 0:
    print('Not Scarcasm')
if res.any() == 1:
    print('Scarcasm')